# Part 1

First, I'll make an Asteroid class. Each instance will store its coordinates and look angles to other asteroids. Then each asteroid can determine the number of other asteroids in view.<br>

When finding the look angle between asteroids, it's possible that float precision issues will cause asteroids at the same angle to appear at slightly different angles, but I'm not sure. I'll round to a few decimal places when I compare angles just in case. I'm also going to arbitrarily choose my angles to be in degrees azimuth (that is, degrees clockwise from North), with North pointing up toward y=0. Basically, aligned with North up the way the graphs are shown in the problem description.

In [1]:
from dataclasses import dataclass, field
from typing import Dict
from math import atan, pi

@dataclass
class Asteroid:
    x: int
    y: int
    detects: int = field(default=0)
    # This really should be Dict[Asteroid, float], but a Jupyter bug (?) causes that to
    # throw an error after the kernel is restarted. Using object works fine.
    angles: Dict[object, float] = field(default_factory=dict)
        
    def __hash__(self):
        """Hash on the (x,y) tuple so Asteroids can act as dict keys."""
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f'Asteroid({self.x}, {self.y})'
    
    def add_angle(self, target):
        """Store look angle to Asteroid target, and cause target to store look angle to this Asteroid.
        
        Look angle is in degrees azimuth (i.e. 0=North, 90=East), with North pointing towards y=0 and
        West pointing towards x=0.
        """
        if target in self.angles:
            pass
        # The coordinate system is weird (0,0 in top left), so manually determine the quadrant the look 
        # direction is in to get angle instead of using atan2 to do it.
        # Positive diffx is looking West, positive diffy is looking North.
        diffx = self.x - target.x
        diffy = self.y - target.y
        if diffx == 0:   # Check 0/180 to avoid divide by 0 error
            if diffy > 0:
                angle = 0.0
            if diffy < 0:
                angle = 180.0
        else:
            # Quadrant 1 (N to E)
            if diffx < 0 and diffy >= 0:
                angle = 90 - (atan(abs(diffy/diffx))*180/pi)
            # Quadrant 2 (E to S)
            elif diffx < 0 and diffy < 0: 
                angle = 90 + (atan(abs(diffy/diffx))*180/pi)
            # Quadrant 3 (S to W)
            elif diffx > 0 and diffy <= 0: 
                angle = 270 - (atan(abs(diffy/diffx))*180/pi)
            # Quadrant 4 (W to N)
            else:                
                angle = 270 + (atan(abs(diffy/diffx))*180/pi)
        self.angles[target] = angle
        #print(f'{target}: diffx={diffx}, diffy={diffy}, angle={angle}')
        if self not in target.angles:
            target.add_angle(self)
    
    @property
    def num_visible(self):
        """Return number of visible asteroids."""
        # Using set() throws out any duplicates, since we can only see one asteroid if two or more
        # are at the same angle. Round to 3 decimal places in case of any float precision issues.
        return len(set(round(v, 3) for v in self.angles.values()))                   

In [2]:
def parse_input(in_str):
    """Return a list of Asteroids from a map string. # is an Asteroid, any other character is empty."""
    asteroids = []
    rows = in_str.split('\n')
    for y, row in enumerate(rows):
        for x, char in enumerate(row):
            if char == '#':
                asteroids.append(Asteroid(x, y))
    return asteroids

Let's test with the first 5x5 example. Asteroids are located at:<br>
(1,0), (4,0), (0,2), (1,2), (2,2), (3,2), (4,2), (4,3), (3,4), (4,4)

In [3]:
ex1_str = """.#..#
.....
#####
....#
...##"""
ex1_rocks = parse_input(ex1_str)
print(ex1_rocks)

[Asteroid(1, 0), Asteroid(4, 0), Asteroid(0, 2), Asteroid(1, 2), Asteroid(2, 2), Asteroid(3, 2), Asteroid(4, 2), Asteroid(4, 3), Asteroid(3, 4), Asteroid(4, 4)]


Check that the angle from (3,4) to (2,2) is equal to the angle from (3,4) to (1,0). Both are in quadrant 4 (northwest-ish), so angle should be `270 + atan(2/1)*180/pi = 333.4 deg`.

In [4]:
rock34 = Asteroid(3, 4)
rock22 = Asteroid(2, 2)
rock10 = Asteroid(1, 0)
rock34.add_angle(rock22)
rock34.add_angle(rock10)
print(rock34.angles)
print(rock34.angles[rock22] == rock34.angles[rock10])

{Asteroid(2, 2): 333.434948822922, Asteroid(1, 0): 333.434948822922}
True


Check the 0, 90, 180, and 270 degree cases to make sure they're handled correctly. Also check that each of the Asteroids is getting the look angle to rock34 correctly added to their angle list.

In [5]:
rock34 = Asteroid(3,4)
rock24 = Asteroid(2,4)    # 270 degrees from rock34, 90 degrees to rock 34
rock44 = Asteroid(4,4)    # 90 degrees from rock34, 270 degrees to rock34
rock35 = Asteroid(3,5)    # 180 degrees from rock34, 0 degres to rock34
rock33 = Asteroid(3,3)    # 0 degrees from rock34, 180 degrees to rock34
for target in [rock24, rock44, rock35, rock33]:
    rock34.add_angle(target)
    print(target.angles)  # Should show 90, 270, 0, 180
print(rock34.angles)      # Should show 270, 90, 180, 0

{Asteroid(3, 4): 90.0}
{Asteroid(3, 4): 270.0}
{Asteroid(3, 4): 0.0}
{Asteroid(3, 4): 180.0}
{Asteroid(2, 4): 270.0, Asteroid(4, 4): 90.0, Asteroid(3, 5): 180.0, Asteroid(3, 3): 0.0}


So all of the look angle calculation and storage looks correct. Now I need to make a function to walk through the Asteroid list and calculate all the look angles between every pair of asteroids. Then I can use that to figure out how many are in view for each asteroid.

In [6]:
from itertools import combinations

def calc_all_angles(asteroids):
    """Calculate look angles between all pairings of 2 asteroids.
    
    Ignores permutations (e.g. get (1,1)->(2,2), but not (2,2)->(1,1)) since Asteroid.add_angle
    adds the look angle to both asteroids.
    This is an in-place operation on the elements of asteroids.
    """
    pairings = combinations(asteroids, 2)
    for pairing in pairings:
        pairing[0].add_angle(pairing[1])

Test `calc_all_angles` by checking the results for Asteroid(3,4). Asteroid(1,0) and (2,2) should have the same angle (333.4 as above), (3,2) should be at 0 degrees, and (4,4) should be at 90 degrees.

In [7]:
calc_all_angles(ex1_rocks)
print(ex1_rocks[-2].angles)

{Asteroid(1, 0): 333.434948822922, Asteroid(4, 0): 14.036243467926468, Asteroid(0, 2): 303.69006752597977, Asteroid(1, 2): 315.0, Asteroid(2, 2): 333.434948822922, Asteroid(3, 2): 0.0, Asteroid(4, 2): 26.56505117707799, Asteroid(4, 3): 45.0, Asteroid(4, 4): 90.0}


Finally, a function to find the asteroid with the most visible asteroids.. 

In [8]:
def max_num_visible(asteroids):
    """Return (asteroid, num_visible) for the asteroid with the most visible asteroids."""
    visible = {asteroid: asteroid.num_visible for asteroid in asteroids}
    max_vis_asteroid = max(visible, key=lambda k: visible[k])
    return max_vis_asteroid, visible[max_vis_asteroid]

For example 1, Asteroid(3,4) should have the most detections with 8.

In [9]:
print(max_num_visible(ex1_rocks))

(Asteroid(3, 4), 8)


Pull all of the above functions together into one function for part 1, then check example 1. The result should be Asteroid(3,4) with 8 detections.

In [10]:
def part1(input_str):
    asteroids = parse_input(input_str)
    calc_all_angles(asteroids)
    print(max_num_visible(asteroids))

part1(ex1_str)

(Asteroid(3, 4), 8)


Check the other examples:

In [11]:
# Example 2: Asteroid(5,8), 33 visible
ex2_str = """......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####"""
part1(ex2_str)

(Asteroid(5, 8), 33)


In [12]:
# Example 3: Asteroid(1,2), 35 visible
ex3_str = """#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###."""
part1(ex3_str)

(Asteroid(1, 2), 35)


In [13]:
# Example 4: Asteroid(6,3), 41 visible
ex4_str = """.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#.."""
part1(ex4_str)

(Asteroid(6, 3), 41)


In [14]:
# Example 5: Asteroid(11,13), 210 visible
ex5_str = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""
part1(ex5_str)

(Asteroid(11, 13), 210)


All the examples check out, so run the problem.

In [15]:
with open('day10_input.txt', 'r') as infile:
    part1_str = infile.read()
part1(part1_str)

(Asteroid(22, 28), 326)


# Part 2

For this part the Asteroids need a couple new capabilities. 

First, I need to be able to sort the other visible Asteroids by angle, ascending from 0 to 360. That will be used to do the clockwise laser sweep and destroy Asteroids as they're encountered. I also need to calculate the distance between Asteroids, so that if there are overlapping Asteroids I can tell which one gets destroyed first by the laser. I'll make a new `targets` dict to track angle and distance per Asteroid.

I also need to track the order in which Asteroids are destroyed, since the puzzle requires knowing the 200th one in particular. Knowing all of them will be helpful for checking the example. I'll make a `destroyed` list for that.

Finally I'll need some functions to add a target asteroid (which will get angle and distance), sort targets by angle and distance, and do the laser firing.

In [16]:
from typing import Tuple
from math import sqrt

@dataclass
class Asteroid2(Asteroid):
    # Elements of targets will be Asteroid: (angle, distance)
    targets: Dict[object, Tuple[float, float]] = field(default_factory=dict)
    destroyed: list = field(default_factory=list)
    
    def add_target(self, target):
        """Add Asteroid target to self.targets as target: (angle, distance)."""
        super().add_angle(target)
        self.targets[target] = (self.angles[target], self.distance(target))
        
    def distance(self, target):
        """Returns the distance from self to target."""
        dx = self.x - target.x
        dy = self.y - target.y
        return sqrt(dx**2 + dy**2)
    
    def sort_targets(self):
        """In-place sort the targets dict by ascending angle and distance.
        
        Asteroids with overlapping angles will be ordered closest to furthest.
        So if targets = {A: (120, 5), B: (38, 6), C: (38, 2)} before sort, 
        after sort it will be targets = {C: (38, 2), B: (38, 6), A: (120, 5)}
        """
        # Sort targets by angle first.
        angle_sort = {k:v for k,v in sorted(self.targets.items(), key=lambda item: item[1][0])}
        # Empty dict for our sorted results
        sorted_dict = {}
        
        # We may have multiple targets at given angle, so we need to sort those targets
        # by ascending distance.        
        # Get a set of the unique angles in ascending order, then iterate through them
        angle_set = sorted(set(v[0] for v in self.targets.values()))
        for angle in angle_set:
            # Storage for overlapping asteroids to be sorted by distance
            overlapping = {}
            # Iterate over a copy of angle_sort.items() because we're going to mutate
            # angle_sort during the loop.
            for k,v in list(angle_sort.items()).copy():
                if v[0] == angle:
                    # If angle matches, add to overlapping to be sorted. Remove from angle_sort
                    # so we don't have to iterate over it on the next angle.
                    overlapping[k] = v
                    del angle_sort[k]
                else:
                    # If we reach a target with a different angle, stop since we've
                    # got all the overlaps at the current angle
                    break
            # Sort overlapping by distance
            overlapping = {k:v for k,v in sorted(overlapping.items(), key=lambda item: item[1][1])}
            
            sorted_dict.update(overlapping)
        
        self.targets = sorted_dict
    
    def fire_ze_laser(self):
        """Add targets to self.destroyed in the order they are destroyed."""
        while len(self.destroyed) < len(self.targets):
            # Do a single pass over the targets list, only destroying the first
            # target in any overlaps
            last_angle = None
            for k,v in self.targets.items():
                if v[0] != last_angle and k not in self.destroyed:
                    self.destroyed.append(k)
                    last_angle = v[0]
    
    def __hash__(self):
        return super().__hash__()
    
    def __repr__(self):
        return f'Asteroid2({self.x}, {self.y})'

Need a new parsing function to make Asteroid2 instances rather than Asteroid instances.

In [17]:
def parse_input_p2(in_str):
    """Return a list of Asteroid2 from a map string. # is an Asteroid, any other character is empty.
    
    Same as part1 parse, but uses the new Asteroid2.
    """
    asteroids = []
    rows = in_str.split('\n')
    for y, row in enumerate(rows):
        for x, char in enumerate(row):
            if char == '#':
                asteroids.append(Asteroid2(x, y))
    return asteroids

For the first example in part 2, the laser station is at (8,3). Check that I get that result using the method from part 1:

In [18]:
p2ex1_str = """.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....#...###..
..#.#.....#....##"""
p2ex1_rocks = parse_input_p2(p2ex1_str)
calc_all_angles(p2ex1_rocks)
p2ex1_station, num_vis = max_num_visible(p2ex1_rocks)
print(p2ex1_station)

Asteroid2(8, 3)


Now that I know the laser station, I need a function to populate and sort the station's target list. I need to take the asteroid list, remove the station, add every other asteroid to the station's target list, then do the angle/distance sort. Then everything will be ready to FIRE ZE LASER.

In [19]:
def assign_targets(asteroid, asteroids):
    """Add all elements of asteroids to asteroid's target list, except itself."""
    # Remove the laser asteroid from the list, then add all the rest as targets
    targets = asteroids.copy()
    targets.remove(asteroid)
    for target in targets:
        asteroid.add_target(target)
    asteroid.sort_targets()

To test, the first 5 asteroids to be destroyed should be: (8,1), (9,0), (9,1), (10,0), (9,2). Printing the station's target list, check the first five asteroids that will be destroyed, remembering that only the first asteroid at a given angle will be destroyed on the first run.

In [20]:
assign_targets(p2ex1_station, p2ex1_rocks)
print(list(p2ex1_station.targets.items())[:10])

[(Asteroid2(8, 1), (0.0, 2.0)), (Asteroid2(8, 0), (0.0, 3.0)), (Asteroid2(9, 0), (18.43494882292201, 3.1622776601683795)), (Asteroid2(9, 1), (26.56505117707799, 2.23606797749979)), (Asteroid2(10, 0), (33.690067525979785, 3.605551275463989)), (Asteroid2(9, 2), (45.0, 1.4142135623730951)), (Asteroid2(10, 1), (45.0, 2.8284271247461903)), (Asteroid2(11, 1), (56.309932474020215, 3.605551275463989)), (Asteroid2(12, 1), (63.43494882292201, 4.47213595499958)), (Asteroid2(14, 0), (63.43494882292201, 6.708203932499369))]


Reading the above: (8,1) is destroyed, (8,0) is skipped, (9,0) is destroyed, (9,1) is destroyed, (10,0) is destroyed, (9,2) is destroyed, and so on. Looks like it's working correctly.

Finally, fire the laser and check the destroyed list against the example text. Starts with (8,1), (9,0), (9,1) and ends with (16,1), (13,3), (14,3).

In [21]:
p2ex1_station.fire_ze_laser()
print(p2ex1_station.destroyed)

[Asteroid2(8, 1), Asteroid2(9, 0), Asteroid2(9, 1), Asteroid2(10, 0), Asteroid2(9, 2), Asteroid2(11, 1), Asteroid2(12, 1), Asteroid2(11, 2), Asteroid2(15, 1), Asteroid2(12, 2), Asteroid2(13, 2), Asteroid2(14, 2), Asteroid2(15, 2), Asteroid2(12, 3), Asteroid2(16, 4), Asteroid2(15, 4), Asteroid2(10, 4), Asteroid2(4, 4), Asteroid2(2, 4), Asteroid2(2, 3), Asteroid2(0, 2), Asteroid2(1, 2), Asteroid2(0, 1), Asteroid2(1, 1), Asteroid2(5, 2), Asteroid2(1, 0), Asteroid2(5, 1), Asteroid2(6, 1), Asteroid2(6, 0), Asteroid2(7, 0), Asteroid2(8, 0), Asteroid2(10, 1), Asteroid2(14, 0), Asteroid2(16, 1), Asteroid2(13, 3), Asteroid2(14, 3)]


Looking good! Now make a function to tie all of the part 2 pieces together and try it on the second example, which is the same map as part 1 example 5. Station is at (11,13).

In [22]:
def part2(input_str):
    asteroids = parse_input_p2(input_str)
    calc_all_angles(asteroids)
    station, num_vis = max_num_visible(asteroids)
    print(f'Station: {station}')
    assign_targets(station, asteroids)
    station.fire_ze_laser()
    return station.destroyed

In [23]:
destroyed = part2(ex5_str)
for i in [1, 2, 3, 10, 20, 50, 100, 199, 200, 201, 299]:
    print(f'#{i} destroyed: {destroyed[i-1]}')

Station: Asteroid2(11, 13)
#1 destroyed: Asteroid2(11, 12)
#2 destroyed: Asteroid2(12, 1)
#3 destroyed: Asteroid2(12, 2)
#10 destroyed: Asteroid2(12, 8)
#20 destroyed: Asteroid2(16, 0)
#50 destroyed: Asteroid2(16, 9)
#100 destroyed: Asteroid2(10, 16)
#199 destroyed: Asteroid2(9, 6)
#200 destroyed: Asteroid2(8, 2)
#201 destroyed: Asteroid2(10, 9)
#299 destroyed: Asteroid2(11, 1)


All correct! Now run the problem:

In [24]:
with open('day10_input.txt', 'r') as infile:
    input_str = infile.read()
destroyed = part2(input_str)
print(f'200th destroyed: {destroyed[199]}')
print(destroyed[199].x*100 + destroyed[199].y)

Station: Asteroid2(22, 28)
200th destroyed: Asteroid2(16, 23)
1623
